# Non static Arbitrage (Triangular)

Imagine we had a list of 10 currencies and we want to check for the possible combinations of paths to get an arbitrage

So: "BTC", "USD", "NGN", "BNB", "ETH", "ARB", "NOT", "MATIC", "EUR", "GBP"

These currencies are the 10 we will try to see if an arbitrage exists between the paths

Just a quick note, we could try to group the currencies into 3 groups and permutate. This could work, but it would need to run every instance of each three groups in other word it would take time to process each group since we want to run it at the same time. 

But Lets first Try something

what is 10!


In [2]:
from itertools import permutations, combinations
import numpy as np
import pandas as pd

currency_list = ["usd","ngn","eur","gbp","btc"]
groups = permutations(currency_list, 2)
r_list = []
e_list = []
for tp in groups:
    if f"{tp[1]+tp[0]}" not in r_list:
        r_list.append(f"{tp[0]+tp[1]}")
    e_list.append( f"{tp[0]+tp[1]}")

pair =  np.array(r_list)
pair, e_list

(array(['usdngn', 'usdeur', 'usdgbp', 'usdbtc', 'ngneur', 'ngngbp',
        'ngnbtc', 'eurgbp', 'eurbtc', 'gbpbtc'], dtype='<U6'),
 ['usdngn',
  'usdeur',
  'usdgbp',
  'usdbtc',
  'ngnusd',
  'ngneur',
  'ngngbp',
  'ngnbtc',
  'eurusd',
  'eurngn',
  'eurgbp',
  'eurbtc',
  'gbpusd',
  'gbpngn',
  'gbpeur',
  'gbpbtc',
  'btcusd',
  'btcngn',
  'btceur',
  'btcgbp'])

# YEHHHHH

I eventually found a way to group that bogus piece of currencies into a smaller piece to work with

In [3]:
# currency_pair_values ={}
# for symbols in final_currency_pairs:
#     if len(currency_pair_values) <= len(final_currency_pairs)*2:

# group = combinations(currency_list, 3)
# unique_elements = set(group)

# currency_pair1 = {f'{elements[0]+elements[1]}' for elements in unique_elements}
# currency_pair2 = {f'{elements[0]+elements[2]}' for elements in unique_elements}
# currency_pair3 = {f'{elements[1]+elements[2]}' for elements in unique_elements} , final_currency_pairs = np.array([currency_pair1.union(currency_pair2).union(currency_pair3)])
# quote_values = np.array({site_data.loc[i]["Quotes"] :value for i,value in enumerate(site_data["Price"])})

In [4]:
site_data = pd.read_csv("Site Data/Custom_quotes2.csv")
df = pd.DataFrame(site_data, columns=["Quotes", "Price","Implied1", "Implied2", "Implied3", "XZ1", "XZ2", "XZ3", "XY1", "XY2", "XY3", "Is_aribitrage1", "Is_aribitrage2", "Is_aribitrage3",  "Profit"])


I want to derive a numpy array of the quoted lists, only their values

In [15]:
# Lets initiate the quotes
derived_quotes  = ["usdngn","eurngn","eurusd","gbpusd","gbpeur","gbpngn","btcusd","btcngn","btceur","btcgbp"]
# this is using of idiomatic expressions to give context to python code while improving its speed
pair_list =  { f"{i}":site_data["Price"].iloc[index] for i in pair for index, value in enumerate(site_data["Quotes"]) if i == value}
quoted_list = np.array([pair_list[i] for i in pair_list])
quoted_list, pair_list

(array([1.45321000e+03, 8.83150000e-01, 7.60090000e-01, 8.89220000e-06,
        6.09000000e-04, 5.23043554e-04, 6.11901146e-09, 8.58690000e-01,
        1.00686479e-05, 1.16988891e-05]),
 {'usdngn': np.float64(1453.21),
  'usdeur': np.float64(0.88315),
  'usdgbp': np.float64(0.76009),
  'usdbtc': np.float64(8.8922e-06),
  'ngneur': np.float64(0.000609),
  'ngngbp': np.float64(0.0005230435540392151),
  'ngnbtc': np.float64(6.1190114620623915e-09),
  'eurgbp': np.float64(0.85869),
  'eurbtc': np.float64(1.0068647851762738e-05),
  'gbpbtc': np.float64(1.169888914047506e-05)})

In [6]:
quote_z,quote_y,quote_x,quote_w,quote_v,quote_u,quote_t,quote_s,quote_r,quote_q = quoted_list
implied_list = np.array([
    np.array([((1/quote_v)/(1/quote_y)), ((1/quote_u)/(1/quote_x)), ((1/quote_t)/(1/quote_w))]), # Implied rates for quote_z
    np.array([(quote_v/(1/quote_z)), ((1/quote_s)/(1/quote_x)), ((1/quote_r)/(1/quote_w))]), # Implied rates for quotes_y
    np.array([(quote_u/(1/quote_z)), ((1/quote_q)/(1/quote_w)), (quote_s/(1/quote_y))]), # Implied rates for quotes_x
    np.array([(quote_t/(1/quote_z)), (quote_q/(1/quote_x)), ((quote_r)/(1/quote_y))]), # Implied rates for quotes_w
    np.array([((1/quote_r)/ (1/quote_t)), ((1/quote_s)/(1/quote_u)), (quote_y/(quote_z))]), # Implied rates for quotes_v
    np.array([((1/quote_q) /(1/quote_t)), (quote_s/(1/quote_v)), (quote_x/quote_z)]), # Implied rates for quotes_u
    np.array([(quote_q/ (1/quote_u)), (quote_r/(1/quote_v)), (quote_w/quote_z)]), # Implied rates for quotes_t
    np.array([((1/quote_q) /(1/quote_r)), (quote_u/(quote_v)), (quote_x/quote_y)]), # Implied rates for quotes_s
    np.array([((quote_q) /(1/quote_s)), (quote_t/(quote_v)), (quote_w/quote_y)]), # Implied rates for quotes_r
    np.array([((quote_r) /(quote_s)), (quote_t/(quote_u)), (quote_w/quote_x)]) # Implied rates for quotes_r
    
]) # YX list
xz_list = np.array([
    np.array([1/quote_v,1/quote_u, 1/quote_t]), # z
    np.array([quote_v, 1/quote_s, 1/quote_r]), # y
    np.array([quote_u, 1/quote_q, quote_s]), # x
    np.array([quote_t, quote_q, (quote_r)]), # w
    np.array([1/quote_r, 1/quote_s, quote_y]), # v
    np.array([1/quote_q, quote_s, quote_x]), # u
    np.array([quote_q, quote_r, quote_w]), # t 
    np.array([1/quote_q, quote_u, quote_x]), # s
    np.array([quote_q, quote_t, quote_w]), # r
    np.array([quote_r, quote_t, quote_w]), # r
    ])
xy_list = np.array([xz_list/implied_list])
implied_df = pd.DataFrame(implied_list)
implied_df

,0,1,2
0,1.450164e+03,1.453206e+03,1.453209e+03
1,8.850049e-01,8.851739e-01,8.831573e-01
2,7.600921e-01,7.600893e-01,7.583521e-01
3,8.892209e-06,8.892209e-06,8.892126e-06
4,6.077292e-04,6.091180e-04,6.077236e-04
5,5.230421e-04,5.229422e-04,5.230421e-04
6,6.119029e-09,6.131807e-09,6.119006e-09
7,8.606499e-01,8.588564e-01,8.606579e-01
8,1.004572e-05,1.004764e-05,1.006873e-05
9,1.172559e-05,1.169886e-05,1.169888e-05


TypeError: unhashable type: 'dict_keys'